# 4-SFT

在预训练阶段后，我们应该能够获得一个下一词预测模型，此时的模型已经掌握了大量的知识。不过，仅仅具备下一词预测能力是不够的，我们希望大模型能够获得问答能力，这一能力便是在有监督微调（Supervised Fine Tuning，SFT）阶段获得的。

在这个笔记本中，我们仅对 SFT 的训练流程进行展示和学习，因此只给出必要的代码片段，如 wandb 和 ddp 不会在此笔记本中涉及。

此笔记本的完整实现见主仓库 `/minimind/train_full_sft.py`

In [1]:
# 导入依赖
import os
import platform
import argparse
import time
import math
import warnings

import pandas as pd
import torch
import torch.nn.functional as F
import torch.distributed as dist
from contextlib import nullcontext

from torch import optim, nn
from torch.nn.parallel import DistributedDataParallel
from torch.utils.data import DataLoader, DistributedSampler
from transformers import AutoTokenizer, AutoModelForCausalLM
from demo.model import MiniMindLM
from demo.LMConfig import LMConfig
from demo.dataset import SFTDataset

In [2]:
warnings.filterwarnings('ignore')

## 可选参数设置

首先，查看训练的可选参数，这些参数在实际使用时通过命令行导入，为了保持笔记本的易用性，选择用 class 进行包装

In [3]:
class args:
    # out_dir: str = "out" # pytorch 格式权重文件保存位置 我们只展示训练过程 所以不使用
    epochs: int = 1 # 训练轮数
    batch_size: int = 2 # pretrain 数据集仅两个样本，设置 batch 为 2
    learning_rate: float = 5e-4 # 学习率
    device: str = 'cuda' if torch.cuda.is_available() else 'cpu'
    dtype: str = 'bfloat16' # 16 bit 浮点数：8 bit 指数 + 7 bit 尾数
    # use_wandb: bool = False # 是否使用 wandb 我们不使用
    wandb_project: str = 'MiniMind-Notebook'
    num_workers: int = 1 # 工作进程数
    # ddp：bool = False # 单机多卡
    accumulation_steps: int = 1 # 梯度累积步数
    grad_clip: float = 1.0 # 梯度剪裁
    warmup_iters: int = 0 # 学习率热启动
    log_interval: int = 1 # 每一步打印日志 仅用于观察
    # save_interval: int = 100 # checkpoint 保存点 我们不使用
    local_rank: int = 1 # device 设备号
    dim: int = 512 # 词嵌入维度 模型超参数
    n_layers: int = 2 # MiniMind Block 数量 模型超参数
    max_seq_len: int = 512 # 序列长度阈值
    use_moe: bool = False # 是否启用混合专家
    data_path: str = './demo/sft_data.jsonl' # 数据集路径

In [4]:
args.device

'cuda'

## 初始化训练

接下来，我们对一些重要模块进行初始化，我们已经了解过，分词器，模型和数据集是大模型的基本组件，我们对其进行初始化。

> 注意 与预训练阶段不同的是 在 sft 阶段 我们实际上是在上一阶段训练获得的模型的基础上修改数据集进行接续训练 因此需要载入上一阶段的模型权重 出于展示的目的 载入权重的代码在此笔记本中只作展示 并不执行

In [5]:
def init_model(lm_config):
    tokenizer = AutoTokenizer.from_pretrained('../model/minimind_tokenizer')
    model = MiniMindLM(lm_config).to(args.device)
    moe_path = '_moe' if lm_config.use_moe else ''
    # ckp = f'./out/pretrain_{lm_config.dim}{moe_path}.pth' # 指示上一阶段训练保存的模型文件位置
    # state_dict = torch.load(ckp, map_location=args.device) # 载入模型状态字典
    # model.laod_state_dict(state_dict, strict=False) # 装入模型
    print(f'LLM总参数量：{sum(p.numel() for p in model.parameters() if p.requires_grad) / 1e6:.3f} 百万')
    model = model.to(args.device)
    return model, tokenizer

In [6]:
lm_config = LMConfig(dim=args.dim, n_layers=args.n_layers, max_seq_len=args.max_seq_len, use_moe=args.use_moe)
model, tokenizer = init_model(lm_config)

train_ds = SFTDataset(args.data_path, tokenizer, max_length=lm_config.max_seq_len)

train_loader = DataLoader(
    train_ds,
    batch_size=args.batch_size,
    pin_memory=True,
    drop_last=False,
    shuffle=False,
    num_workers=args.num_workers,
)

print(model.device, tokenizer.vocab_size, train_loader)

LLM总参数量：8.915 百万
cuda:0 6400 <torch.utils.data.dataloader.DataLoader object at 0x00000184B5C6B4F0>


In [7]:
loader = iter(train_loader)
next(loader), len(train_ds), len(loader)

([tensor([[  1,  85, 736,  ...,   0,   0,   0],
          [  1,  85, 736,  ...,   0,   0,   0]]),
  tensor([[ 85, 736, 201,  ...,   0,   0,   0],
          [ 85, 736, 201,  ...,   0,   0,   0]]),
  tensor([[0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0]])],
 2,
 1)

我们发现，train loader 的每一个 iter 都包含一个长度为 3 的张量列表，这是因为 train_dataset 每一次取数据都会返回三个张量，分别为:

- 样本 X: 包含 \<bos> 在内的输入 conversation
- 标签 Y: 包含 \<eos> 在内的输出 conversation
- 掩码 loss_mask: 指示需要计算损失的 token 位置

由于我们的数据集只有两条数据，而 batch size 设置为 2，因此我们的 dataloader 只有一个 iter.

## 启动训练

训练一个深度学习模型，还涉及到了优化器，损失函数和学习率调度。接下来，我们查看 MiniMind 训练部分的代码，并进行一轮简单的训练。

> 不难发现 pretrain 阶段和 sft 阶段的训练主体差不多 因为这两个阶段的差异体现在数据集格式 而数据集在经过 chat template 格式化后差异小了很多

In [8]:
# 学习率调度方面 采用余弦退火学习率
def get_lr(current_step, total_steps, lr):
    return lr / 10 + 0.5 * lr * (1 + math.cos(math.pi * current_step / total_steps))

# 优化器方面 选择 AdamW 优化器 并在混精度场景下创建 scaler 进行梯度缩放避免数值下溢
scaler = torch.cuda.amp.GradScaler(enabled=(args.dtype in ['float16', 'bfloat16']))
optimizer = optim.AdamW(model.parameters(), lr=args.learning_rate)

device_type = "cuda" if "cuda" in args.device else "cpu"
ctx = nullcontext() if device_type == "cpu" else torch.cuda.amp.autocast() # 在 cuda 上启动混精度训练，否则空白上下文

接下来，我们来看看 MiniMind 的训练函数

In [9]:
def train_epoch(epoch):
    loss_fct = nn.CrossEntropyLoss(reduction='none') # 损失函数 采用交叉熵损失
    start_time = time.time()
    for step, (X, Y, loss_mask) in enumerate(train_loader):
        X = X.to(args.device)
        Y = Y.to(args.device)
        loss_mask = loss_mask.to(args.device)

        lr = get_lr(epoch * iter_per_epoch + step, args.epochs * iter_per_epoch, args.learning_rate)
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

        with ctx:
            res = model(X) # 前向推理
            loss = loss_fct(
                res.logits.view(-1, res.logits.size(-1)),
                Y.view(-1)
            ).view(Y.size()) # 取生成的最后一个 token 的 logits 计算损失
            loss = (loss * loss_mask).sum() / loss_mask.sum()
            loss += res.aux_loss # 若为混合专家 则将 MOE 辅助损失纳入考虑
            loss = loss / args.accumulation_steps # 梯度累积

        scaler.scale(loss).backward() # 梯度缩放

        if (step + 1) % args.accumulation_steps == 0:
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), args.grad_clip) # 梯度剪裁

            scaler.step(optimizer)
            scaler.update()

            optimizer.zero_grad(set_to_none=True) # 单步更新

        if step % args.log_interval == 0:
            spend_time = time.time() - start_time
            print(
                'Epoch:[{}/{}]({}/{}) loss:{:.3f} lr:{:.12f} epoch_Time:{}min:'.format(
                    epoch + 1,
                    args.epochs,
                    step,
                    iter_per_epoch,
                    loss.item() * args.accumulation_steps,
                    optimizer.param_groups[-1]['lr'],
                    spend_time / (step + 1) * iter_per_epoch // 60 - spend_time // 60))

        # 到达指定保存步数时，save as PyTorch
        # if (step + 1) % args.save_interval == 0 and (not ddp or dist.get_rank() == 0):
        #     model.eval()
        #     moe_path = '_moe' if lm_config.use_moe else ''
        #     ckp = f'{args.save_dir}/pretrain_{lm_config.dim}{moe_path}.pth'

        #     if isinstance(model, torch.nn.parallel.DistributedDataParallel):
        #         state_dict = model.module.state_dict()
        #     else:
        #         state_dict = model.state_dict()

        #     torch.save(state_dict, ckp)
        #     model.train()

准备完毕，我们进行一轮长度 1 个 iter 的训练看看怎么个事。

In [10]:
iter_per_epoch = len(train_loader)
for epoch in range(args.epochs):
    train_epoch(epoch)

Epoch:[1/1](0/1) loss:8.865 lr:0.000550000000 epoch_Time:0.0min:
